In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122796")
exp = Experiment(workspace=ws, name="Udacity-project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122796
Azure region: southcentralus
Subscription id: 8ca4d354-cab8-4c39-9d61-ea227d2daaad
Resource group: aml-quickstarts-122796


In [2]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

ws = Workspace.from_config() # this automatically looks for a directory .azureml

#Create compute cluster
cluster_name="Udacity-project1"

try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                       max_nodes=4, 
                                                       idle_seconds_before_scaledown=2400,
                                                       vm_priority='lowpriority')
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_target.wait_for_completion(show_output=True)
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Udacity-project ComputeInstance Succeeded
Udacity-project1 AmlCompute Succeeded


## Using Hyperdrive

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(parameter_space={"--C":choice(0.001,0.01,0.1,0.5,1,10,100,1000),"--max_iter": choice(25,50,100,150,200,250,300)})

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 5, slack_factor = 0.1, delay_evaluation = 5) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',compute_target='Udacity-project1',entry_script='train.py')
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs= 50,
                             max_concurrent_runs=4)
### YOUR CODE HERE ###

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

Hyperdrive_run = exp.submit(config=hyperdrive_config)
### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [5]:
RunDetails(Hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
Hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_5e30f476-f6a8-4d92-b6c2-671cc391cc43
Web View: https://ml.azure.com/experiments/Udacity-project1/runs/HD_5e30f476-f6a8-4d92-b6c2-671cc391cc43?wsid=/subscriptions/8ca4d354-cab8-4c39-9d61-ea227d2daaad/resourcegroups/aml-quickstarts-122796/workspaces/quick-starts-ws-122796

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-10-25T11:38:38.005815][API][INFO]Experiment created<END>\n""<START>[2020-10-25T11:38:38.369733][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-10-25T11:38:38.597633][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-10-25T11:38:39.1336913Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_5e30f476-f6a8-4d92-b6c2-671cc391cc43
Web View: https://ml.azure.com/experiments/Udacity-project1/runs/HD_5e30f476-f6a8-4d92-b6c2-671cc391cc43?wsid=/subscriptions/8ca

{'runId': 'HD_5e30f476-f6a8-4d92-b6c2-671cc391cc43',
 'target': 'Udacity-project1',
 'status': 'Canceled',
 'startTimeUtc': '2020-10-25T11:38:37.646587Z',
 'endTimeUtc': '2020-10-25T11:50:12.441191Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/8ca4d354-cab8-4c39-9d61-ea227d2daaad/resourceGroups/aml-quickstarts-122796/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-122796/experiments/**SCRUBBED**/runs/HD_5e30f476-f6a8-4d92-b6c2-671cc391cc43_0, {\'error\': {\n    "additional_properties": {\n        "componentName": "execution-worker"\n    },\n    "error": {\n        "additional_properties": {\n            "referenceCode": null,\n            "messageFormat": null,\n            "severity": null,\n            "messageParameters": null\n        },\n        "code": "UserError",\n        "m

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = Hyperdrive_run.get_best_run_by_primary_metric()
best_run

In [8]:
parameter_values = best_run.get_details()['runDefinition']['arguments']
print(parameter_values)

AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
best_run_metrics = best_run.get_metrics()
print("Best_run_id",best_run.id)
print("Best_run_accuracy",best_run_metrics['Accuracy'])

In [ ]:
best_run.get_file_names()

In [ ]:
# Registering the best model for future usage
best_model=best_run.register_model(model_name='Hyperdrive_Model',model_path='outputs/Best_Model.joblib')
best_model

## Using AutoML

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
df = TabularDatasetFactory.from_delimited_files(path=url_path)
### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)
print(type(x))
print(type(y))
      ### YOUR DATA OBJECT HERE ###)

In [ ]:
df_y = pd.DataFrame(y,columns=['y'])
df_y.head()

In [ ]:
final_df = pd.concat([x,df_y],axis=1)
final_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

#exporting the dataFrame to csv
final_df.to_csv('training/train_data.csv')

In [ ]:
datastore=ws.get_default_datastore()  # checking the Default Work space storage location.
datastore

In [ ]:
datastore.upload(src_dir='training/',
               target_path='data/train.csv',
               overwrite=True, show_progress=True)

In [ ]:
from azureml.core import Workspace, Datastore, Dataset
train_data=Dataset.Tabular.from_delimited_files(path=[(datastore, 'data/train.csv')])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data= train_data,
    label_column_name= 'y',
    n_cross_validations=5,
    compute_target=compute_target, 
    iterations= 100,
    max_concurrent_iterations=8,
    max_cores_per_iteration=-1)

In [ ]:
# Submit your automl run
### YOUR CODE HERE ###
from azureml.widgets import RunDetails
automl_run = exp.submit(automl_config, show_output=True)

In [ ]:
RunDetails(automl_run).show()

In [ ]:
automl_run.wait_for_completion(show_output=True)

In [ ]:
automl_run.get_metrics('accuracy')

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run,model=automl_run.get_output()

In [ ]:
#saving the model.
import joblib
best_model_automl=joblib.dump(model,filename='best_model.joblib')

In [ ]:
best_model_automl

In [ ]:
model

In [ ]:
model._final_estimator

In [ ]:
## Deleting the Compute Cluster

compute_target.delete()